In [1]:
import pandas as pd
from datetime import datetime
from plyer import notification
import requests
import sqlite3
import json

In [2]:
conn = sqlite3.connect('coderhouse.db' )

Função responsável por salvar o DataFrame no Banco de dados local.

In [3]:
def salva_bd(df, nome_tabela):
    conn = sqlite3.connect('coderhouse.db')
    df.to_sql(nome_tabela,conn,if_exists='replace',index = False)
    conn.close()
    return None

Função responsável por carregar o DataFrame do banco de dados local.

In [4]:
def carrega_bd(nome_tabela):
    conn = sqlite3.connect('coderhouse.db')
    query = f"select * from {nome_tabela}"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

Função responsável por listar todas as tabelas salvas no banco de dados local.

In [5]:
def lista_tabelas():
    conn = sqlite3.connect('coderhouse.db')
    query = "select name from sqlite_master where type='table'"
    schema = pd.read_sql(query, conn)
    conn.close()
    return schema

In [6]:
urls = [ 
    'https://brasilapi.com.br/api/banks/v1', #bancos ok
    'https://brasilapi.com.br/api/cvm/corretoras/v1', #corretoras ok
    'https://restcountries.com/v3.1/all' #rest countries ok
]

Função responsável por emitir um alerta caso a requisição da API retorne falha.

In [7]:
def Alerta(nivel, base, etapa):
    
    data_atual = datetime.now()
    data_formatada = data_atual.strftime('%d/%m/%d')
    mensagem = (f'Falha no carregamento da base {base} na etapa {etapa} em {data_formatada}')
    title = nivel
    notification.notify(title = title, message= mensagem, timeout= 10)

Função responsável por direcionar as APIs em seus respectivos Data Frames.

In [8]:
def Extraçao():

    global df_bancos, df_corretoras, df_resties

    for url in urls:
        request = requests.get(url)
        response_code = request.status_code
        if response_code == 200:
            if url == 'https://brasilapi.com.br/api/banks/v1':
                df_bancos = pd.DataFrame(request.json())
                
            elif url == 'https://brasilapi.com.br/api/cvm/corretoras/v1':
                df_corretoras = pd.DataFrame(request.json())
                
            elif url == 'https://restcountries.com/v3.1/all':
                df_resties = pd.DataFrame(request.json())
                
            else:
                print('URL fora da base de dados.')
        else:
            nivel = 'ATENÇÃO: Erro Grave'
            Alerta(nivel = nivel, base = 'APIs', etapa = 'Extraçao')

In [9]:
Extraçao()

                                           INICIANDO TRATAMENTO DO DF_BANCOS

In [10]:
df_bancos.head(2)

,ispb,name,code,fullName
0,00000000,BCO DO BRASIL S.A.,1.0,Banco do Brasil S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.


In [11]:
df_bancos_mod = df_bancos.copy() #mantendo o df original sem que o mesmo seja alterado pelas modificações posteriores

In [12]:
df_bancos_mod.head(2)

,ispb,name,code,fullName
0,00000000,BCO DO BRASIL S.A.,1.0,Banco do Brasil S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.


In [13]:
df_bancos_mod['code'].fillna(0, inplace=True)

In [14]:
df_bancos_mod['code'] = df_bancos_mod['code'].astype(int)

In [15]:
colunas_com_nan = ['name', 'fullName']

In [16]:
df_bancos_mod = df_bancos_mod.dropna(subset=colunas_com_nan)

In [17]:
df_bancos_mod['fullName'] = df_bancos_mod['fullName'].str.upper()

In [18]:
df_bancos_mod['name'] = df_bancos_mod['name'].str.upper()

In [19]:
df_bancos_mod.rename(columns={'ispb': 'ISPB'}, inplace=True)
df_bancos_mod.rename(columns={'name': 'SOCIAL'}, inplace=True)
df_bancos_mod.rename(columns={'code': 'CÓDIGO'}, inplace=True)
df_bancos_mod.rename(columns={'fullName': 'RAZÃO SOCIAL'}, inplace=True)

In [20]:
df_bancos_mod.head(4)

,ISPB,SOCIAL,CÓDIGO,RAZÃO SOCIAL
0,00000000,BCO DO BRASIL S.A.,1,BANCO DO BRASIL S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70,BRB - BANCO DE BRASILIA S.A.
2,00038121,SELIC,0,BANCO CENTRAL DO BRASIL - SELIC
3,00038166,BACEN,0,BANCO CENTRAL DO BRASIL


SALVANDO E ENVIANDO O DF PARA O BANCO DE DADOS LOCAL

In [21]:
salva_bd(df_bancos_mod,'bancos')

In [22]:
lista_tabelas()

,name
0,restaurantes
1,bancos


In [23]:
bancos_teste = carrega_bd('bancos')

In [24]:
bancos_teste.head(2)

,ISPB,SOCIAL,CÓDIGO,RAZÃO SOCIAL
0,00000000,BCO DO BRASIL S.A.,1,BANCO DO BRASIL S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70,BRB - BANCO DE BRASILIA S.A.


                                       INICIANDO TRATAMENTO DO DF_CORRETORAS

In [25]:
df_corretoras.head(2)

,cnpj,type,nome_social,nome_comercial,status,email,telefone,cep,pais,uf,municipio,bairro,complemento,logradouro,data_patrimonio_liquido,valor_patrimonio_liquido,codigo_cvm,data_inicio_situacao,data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,2005-12-31,4228660.18,2275,2006-10-05,1968-01-15
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",2002-12-31,0.00,3514,2002-10-14,2002-10-14


In [26]:
df_corretoras_mod = df_corretoras.copy()

In [27]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cnpj                      373 non-null    object
 1   type                      373 non-null    object
 2   nome_social               373 non-null    object
 3   nome_comercial            373 non-null    object
 4   status                    373 non-null    object
 5   email                     373 non-null    object
 6   telefone                  373 non-null    object
 7   cep                       373 non-null    object
 8   pais                      373 non-null    object
 9   uf                        373 non-null    object
 10  municipio                 373 non-null    object
 11  bairro                    373 non-null    object
 12  complemento               373 non-null    object
 13  logradouro                373 non-null    object
 14  data_patrimonio_liquido   

In [28]:
df_corretoras_mod['data_patrimonio_liquido']

0      2005-12-31
1      2002-12-31
2      1989-12-31
3      2022-12-31
4      2000-12-31
          ...    
368    2017-12-31
369    2017-12-31
370    2022-12-31
371    1996-12-31
372    2022-12-31
Name: data_patrimonio_liquido, Length: 373, dtype: object

In [29]:
df_corretoras_mod['data_patrimonio_liquido'] = pd.to_datetime(df_corretoras_mod['data_patrimonio_liquido'])

In [30]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   cnpj                      373 non-null    object        
 1   type                      373 non-null    object        
 2   nome_social               373 non-null    object        
 3   nome_comercial            373 non-null    object        
 4   status                    373 non-null    object        
 5   email                     373 non-null    object        
 6   telefone                  373 non-null    object        
 7   cep                       373 non-null    object        
 8   pais                      373 non-null    object        
 9   uf                        373 non-null    object        
 10  municipio                 373 non-null    object        
 11  bairro                    373 non-null    object        
 12  complemento           

In [31]:
df_corretoras_mod['data_patrimonio_liquido'] = df_corretoras_mod['data_patrimonio_liquido'].dt.strftime('%d/%m/%Y')

In [32]:
df_corretoras_mod['data_inicio_situacao']

0      2006-10-05
1      2002-10-14
2      1990-06-12
3      2022-06-20
4      2001-04-11
          ...    
368    2018-08-27
369    2019-09-24
370    1967-12-06
371    1997-02-21
372    1998-02-10
Name: data_inicio_situacao, Length: 373, dtype: object

In [33]:
df_corretoras_mod['data_inicio_situacao'] = pd.to_datetime(df_corretoras_mod['data_inicio_situacao'])

In [34]:
df_corretoras_mod['data_inicio_situacao'] = df_corretoras_mod['data_inicio_situacao'].dt.strftime('%d/%m/%Y')

In [35]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cnpj                      373 non-null    object
 1   type                      373 non-null    object
 2   nome_social               373 non-null    object
 3   nome_comercial            373 non-null    object
 4   status                    373 non-null    object
 5   email                     373 non-null    object
 6   telefone                  373 non-null    object
 7   cep                       373 non-null    object
 8   pais                      373 non-null    object
 9   uf                        373 non-null    object
 10  municipio                 373 non-null    object
 11  bairro                    373 non-null    object
 12  complemento               373 non-null    object
 13  logradouro                373 non-null    object
 14  data_patrimonio_liquido   

In [36]:
df_corretoras_mod['data_registro'] = pd.to_datetime(df_corretoras_mod['data_registro'])

In [37]:
df_corretoras_mod['data_registro'] = df_corretoras_mod['data_registro'].dt.strftime('%d/%m/%Y') 

In [38]:
df_corretoras_mod.head(2)

,cnpj,type,nome_social,nome_comercial,status,email,telefone,cep,pais,uf,municipio,bairro,complemento,logradouro,data_patrimonio_liquido,valor_patrimonio_liquido,codigo_cvm,data_inicio_situacao,data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,31/12/2005,4228660.18,2275,05/10/2006,15/01/1968
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",31/12/2002,0.00,3514,14/10/2002,14/10/2002


In [39]:
df_corretoras_mod.columns = [col.capitalize() for col in df_corretoras.columns]

In [40]:
df_corretoras_mod.rename(columns={'Cnpj': 'CNPJ'}, inplace=True)

In [41]:
df_corretoras_mod.rename(columns={'Uf': 'UF'}, inplace=True)

In [42]:
df_corretoras_mod.rename(columns={'Cep': 'CEP'}, inplace=True)

In [43]:
df_corretoras_mod.rename(columns={'Email': 'E-mail'}, inplace=True)

In [44]:
df_corretoras_mod.rename(columns={'Pais': 'País'}, inplace=True)

In [46]:
df_corretoras_mod.rename(columns={'Type': 'Tipo'}, inplace=True)

In [47]:
df_corretoras_mod.head(2)

,CNPJ,Tipo,Nome_social,Nome_comercial,Status,E-mail,Telefone,CEP,País,UF,Municipio,Bairro,Complemento,Logradouro,Data_patrimonio_liquido,Valor_patrimonio_liquido,Codigo_cvm,Data_inicio_situacao,Data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,31/12/2005,4228660.18,2275,05/10/2006,15/01/1968
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",31/12/2002,0.00,3514,14/10/2002,14/10/2002


In [48]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   CNPJ                      373 non-null    object
 1   Tipo                      373 non-null    object
 2   Nome_social               373 non-null    object
 3   Nome_comercial            373 non-null    object
 4   Status                    373 non-null    object
 5   E-mail                    373 non-null    object
 6   Telefone                  373 non-null    object
 7   CEP                       373 non-null    object
 8   País                      373 non-null    object
 9   UF                        373 non-null    object
 10  Municipio                 373 non-null    object
 11  Bairro                    373 non-null    object
 12  Complemento               373 non-null    object
 13  Logradouro                373 non-null    object
 14  Data_patrimonio_liquido   

In [49]:
linhas_com_missing = df_corretoras_mod[df_corretoras_mod.isna().any(axis=1)]

In [50]:
linhas_com_missing

,CNPJ,Tipo,Nome_social,Nome_comercial,Status,E-mail,Telefone,CEP,País,UF,Municipio,Bairro,Complemento,Logradouro,Data_patrimonio_liquido,Valor_patrimonio_liquido,Codigo_cvm,Data_inicio_situacao,Data_registro
18,99988065000154,CORRETORAS,ALVES FERREIRA CV LTDA,,CANCELADA,,,80020,BRASIL,PR,CURITIBA,,,"RUA MAL.DEODORO, 235 - 2 ANDAR",NaN,,2747,29/04/1993,27/12/1991
70,12865507000197,CORRETORAS,"BRICKELL S.A. CRÉDITO, FINANCIAMENTO E INVESTI...",--,CANCELADA,brickellcfi@brickellcfi.com.br,21989735,4534002,BRASIL,SP,SÃO PAULO,ITAIM BIBI,SALA 1302 E1303,"R JOAQUIM FLORIANO, 466",NaN,,3492,23/01/1995,28/03/1988
229,36864992000142,CORRETORAS,MAF DISTRIBUIDORA DE TITULOS E VALORES MOBILIA...,MAF DTVM,CANCELADA,juridico.admfiduciaria@apexgroup.com,45601000,22250040,,RJ,RIO DE JANEIRO,BOTAFOGO,TORRE PÃO DE AÇUCAR,"PRAIA DE BOTAFOGO, 501/ 5º ANDAR PARTE",NaN,,3751,06/11/2020,06/11/2020


In [51]:
df_corretoras_mod = df_corretoras_mod.dropna()

In [52]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
Index: 370 entries, 0 to 372
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   CNPJ                      370 non-null    object
 1   Tipo                      370 non-null    object
 2   Nome_social               370 non-null    object
 3   Nome_comercial            370 non-null    object
 4   Status                    370 non-null    object
 5   E-mail                    370 non-null    object
 6   Telefone                  370 non-null    object
 7   CEP                       370 non-null    object
 8   País                      370 non-null    object
 9   UF                        370 non-null    object
 10  Municipio                 370 non-null    object
 11  Bairro                    370 non-null    object
 12  Complemento               370 non-null    object
 13  Logradouro                370 non-null    object
 14  Data_patrimonio_liquido   370 n

In [53]:
df_corretoras_mod.head(5)

,CNPJ,Tipo,Nome_social,Nome_comercial,Status,E-mail,Telefone,CEP,País,UF,Municipio,Bairro,Complemento,Logradouro,Data_patrimonio_liquido,Valor_patrimonio_liquido,Codigo_cvm,Data_inicio_situacao,Data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,31/12/2005,4228660.18,2275,05/10/2006,15/01/1968
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",31/12/2002,0.00,3514,14/10/2002,14/10/2002
2,10664027000132,CORRETORAS,ABERTURA CCVM LTDA,ABERTURA CCVM LTDA,CANCELADA,,,50010240,BRASIL,PE,RECIFE,,,R DO IMP.D.PEDRO II 239/CJ.102,31/12/1989,5995252.29,329,12/06/1990,08/07/1986
3,44527444000155,CORRETORAS,ABN AMRO CORRETORA DE T-TULOS E VALORES MOBILI...,,EM FUNCIONAMENTO NORMAL,canal.comunicacao@br.abnamro.com,30737417,4551010,,SP,SÃO PAULO,VILA OLIMPIA,"11¦ ANDAR, CJ. 111","RUA FIDÛNCIO RAMOS, N¦ 302, TORRE B",31/12/2022,3729687.41,3794,20/06/2022,20/06/2022
4,33819590000138,CORRETORAS,AÇÃO S.A. CORRETORA DE VALORES E CÂMBIO,ACAO SA CVC,CANCELADA,,5095755,20040040,BRASIL,RJ,RIO DE JANEIRO,CENTRO,,"PRAÇA OLAVO BILAC, 28/ SL. 608",31/12/2000,291862.38,1112,11/04/2001,19/04/1968


SALVANDO E ENVIANDO O DF PARA O BANCO DE DADOS LOCAL

In [54]:
salva_bd(df_corretoras_mod,'corretoras')

In [55]:
lista_tabelas()

,name
0,restaurantes
1,bancos
2,corretoras


In [56]:
corretora_teste = carrega_bd('corretoras')

In [57]:
corretora_teste.head(2)

,CNPJ,Tipo,Nome_social,Nome_comercial,Status,E-mail,Telefone,CEP,País,UF,Municipio,Bairro,Complemento,Logradouro,Data_patrimonio_liquido,Valor_patrimonio_liquido,Codigo_cvm,Data_inicio_situacao,Data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,31/12/2005,4228660.18,2275,05/10/2006,15/01/1968
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",31/12/2002,0.00,3514,14/10/2002,14/10/2002


                                         INICIANDO TRATAMENTO DO DF_RESTIES

In [58]:
df_resties.head(2)

,name,tld,cca2,ccn3,cca3,independent,status,unMember,currencies,idd,...,continents,flags,coatOfArms,startOfWeek,capitalInfo,postalCode,borders,cioc,gini,fifa
0,"{'common': 'French Polynesia', 'official': 'Fr...",[.pf],PF,258,PYF,False,officially-assigned,False,"{'XPF': {'name': 'CFP franc', 'symbol': '₣'}}","{'root': '+6', 'suffixes': ['89']}",...,[Oceania],"{'png': 'https://flagcdn.com/w320/pf.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [-17.53, -149.56]}","{'format': '#####', 'regex': '^((97|98)7\d{2})$'}",NaN,NaN,NaN,NaN
1,"{'common': 'Saint Martin', 'official': 'Saint ...","[.fr, .gp]",MF,663,MAF,False,officially-assigned,False,"{'EUR': {'name': 'Euro', 'symbol': '€'}}","{'root': '+5', 'suffixes': ['90']}",...,[North America],"{'png': 'https://flagcdn.com/w320/mf.png', 'sv...",{},monday,"{'latlng': [18.07, -63.08]}",{'format': '### ###'},[SXM],NaN,NaN,NaN


In [59]:
df_resties_mod = df_resties.copy()

In [60]:
df_resties_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 35 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          250 non-null    object 
 1   tld           249 non-null    object 
 2   cca2          250 non-null    object 
 3   ccn3          249 non-null    object 
 4   cca3          250 non-null    object 
 5   independent   249 non-null    object 
 6   status        250 non-null    object 
 7   unMember      250 non-null    bool   
 8   currencies    247 non-null    object 
 9   idd           250 non-null    object 
 10  capital       246 non-null    object 
 11  altSpellings  250 non-null    object 
 12  region        250 non-null    object 
 13  subregion     245 non-null    object 
 14  languages     249 non-null    object 
 15  translations  250 non-null    object 
 16  latlng        250 non-null    object 
 17  landlocked    250 non-null    bool   
 18  area          250 non-null    

In [61]:
valores_ausentes_por_coluna = df_resties_mod.isna().sum()

In [62]:
print(valores_ausentes_por_coluna)

name              0
tld               1
cca2              0
ccn3              1
cca3              0
independent       1
status            0
unMember          0
currencies        3
idd               0
capital           4
altSpellings      0
region            0
subregion         5
languages         1
translations      0
latlng            0
landlocked        0
area              0
demonyms          1
flag              0
maps              0
population        0
car               0
timezones         0
continents        0
flags             0
coatOfArms        0
startOfWeek       0
capitalInfo       0
postalCode      101
borders          85
cioc             44
gini             83
fifa             44
dtype: int64


In [63]:
df_resties_mod.columns = [col.capitalize() for col in df_resties_mod.columns]

In [64]:
df_resties_mod.head(2)

,Name,Tld,Cca2,Ccn3,Cca3,Independent,Status,Unmember,Currencies,Idd,...,Continents,Flags,Coatofarms,Startofweek,Capitalinfo,Postalcode,Borders,Cioc,Gini,Fifa
0,"{'common': 'French Polynesia', 'official': 'Fr...",[.pf],PF,258,PYF,False,officially-assigned,False,"{'XPF': {'name': 'CFP franc', 'symbol': '₣'}}","{'root': '+6', 'suffixes': ['89']}",...,[Oceania],"{'png': 'https://flagcdn.com/w320/pf.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [-17.53, -149.56]}","{'format': '#####', 'regex': '^((97|98)7\d{2})$'}",NaN,NaN,NaN,NaN
1,"{'common': 'Saint Martin', 'official': 'Saint ...","[.fr, .gp]",MF,663,MAF,False,officially-assigned,False,"{'EUR': {'name': 'Euro', 'symbol': '€'}}","{'root': '+5', 'suffixes': ['90']}",...,[North America],"{'png': 'https://flagcdn.com/w320/mf.png', 'sv...",{},monday,"{'latlng': [18.07, -63.08]}",{'format': '### ###'},[SXM],NaN,NaN,NaN


In [65]:
df_resties_mod['Name'] = [pais['common']for pais in df_resties_mod['Name']]

In [66]:
df_resties_mod['Postalcode']

0      {'format': '#####', 'regex': '^((97|98)7\d{2})$'}
1                                  {'format': '### ###'}
2               {'format': '####', 'regex': '^(\d{4})$'}
3      {'format': '#####', 'regex': '^((97|98)(4|7|8)...
4      {'format': 'CP ####', 'regex': '^(?:CP)*(\d{4}...
                             ...                        
245                                                  NaN
246                                                  NaN
247                                                  NaN
248                                                  NaN
249    {'format': '@####@', 'regex': '^([a-zA-Z]\d{4}...
Name: Postalcode, Length: 250, dtype: object

In [67]:
df_resties_mod = df_resties_mod.drop('Postalcode', axis=1)

In [68]:
pd.options.display.max_columns = None

In [69]:
df_resties_mod.head(2)

,Name,Tld,Cca2,Ccn3,Cca3,Independent,Status,Unmember,Currencies,Idd,Capital,Altspellings,Region,Subregion,Languages,Translations,Latlng,Landlocked,Area,Demonyms,Flag,Maps,Population,Car,Timezones,Continents,Flags,Coatofarms,Startofweek,Capitalinfo,Borders,Cioc,Gini,Fifa
0,French Polynesia,[.pf],PF,258,PYF,False,officially-assigned,False,"{'XPF': {'name': 'CFP franc', 'symbol': '₣'}}","{'root': '+6', 'suffixes': ['89']}",[Papeetē],"[PF, Polynésie française, French Polynesia, Pō...",Oceania,Polynesia,{'fra': 'French'},"{'ara': {'official': 'بولينزيا الفرنسية', 'com...","[17.6797, 149.4068]",False,4167.0,"{'eng': {'f': 'French Polynesian', 'm': 'Frenc...",🇵🇫,{'googleMaps': 'https://goo.gl/maps/xgg6BQTRye...,280904,"{'signs': ['F'], 'side': 'right'}","[UTC-10:00, UTC-09:30, UTC-09:00]",[Oceania],"{'png': 'https://flagcdn.com/w320/pf.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [-17.53, -149.56]}",NaN,NaN,NaN,NaN
1,Saint Martin,"[.fr, .gp]",MF,663,MAF,False,officially-assigned,False,"{'EUR': {'name': 'Euro', 'symbol': '€'}}","{'root': '+5', 'suffixes': ['90']}",[Marigot],"[MF, Collectivity of Saint Martin, Collectivit...",Americas,Caribbean,{'fra': 'French'},"{'ara': {'official': 'سانت مارتن', 'common': '...","[18.0708, 63.0501]",False,53.0,"{'eng': {'f': 'Saint Martin Islander', 'm': 'S...",🇲🇫,{'googleMaps': 'https://goo.gl/maps/P9ho9QuJ9E...,38659,"{'signs': ['F'], 'side': 'right'}",[UTC-04:00],[North America],"{'png': 'https://flagcdn.com/w320/mf.png', 'sv...",{},monday,"{'latlng': [18.07, -63.08]}",[SXM],NaN,NaN,NaN


In [70]:
df_resties_mod['Currencies']

0          {'XPF': {'name': 'CFP franc', 'symbol': '₣'}}
1               {'EUR': {'name': 'Euro', 'symbol': '€'}}
2      {'VES': {'name': 'Venezuelan bolívar soberano'...
3               {'EUR': {'name': 'Euro', 'symbol': '€'}}
4      {'USD': {'name': 'United States dollar', 'symb...
                             ...                        
245    {'USD': {'name': 'United States dollar', 'symb...
246    {'BWP': {'name': 'Botswana pula', 'symbol': 'P'}}
247    {'PAB': {'name': 'Panamanian balboa', 'symbol'...
248    {'XAF': {'name': 'Central African CFA franc', ...
249    {'USD': {'name': 'United States dollar', 'symb...
Name: Currencies, Length: 250, dtype: object

In [71]:
df_resties_mod.head(1)

,Name,Tld,Cca2,Ccn3,Cca3,Independent,Status,Unmember,Currencies,Idd,Capital,Altspellings,Region,Subregion,Languages,Translations,Latlng,Landlocked,Area,Demonyms,Flag,Maps,Population,Car,Timezones,Continents,Flags,Coatofarms,Startofweek,Capitalinfo,Borders,Cioc,Gini,Fifa
0,French Polynesia,[.pf],PF,258,PYF,False,officially-assigned,False,"{'XPF': {'name': 'CFP franc', 'symbol': '₣'}}","{'root': '+6', 'suffixes': ['89']}",[Papeetē],"[PF, Polynésie française, French Polynesia, Pō...",Oceania,Polynesia,{'fra': 'French'},"{'ara': {'official': 'بولينزيا الفرنسية', 'com...","[17.6797, 149.4068]",False,4167.0,"{'eng': {'f': 'French Polynesian', 'm': 'Frenc...",🇵🇫,{'googleMaps': 'https://goo.gl/maps/xgg6BQTRye...,280904,"{'signs': ['F'], 'side': 'right'}","[UTC-10:00, UTC-09:30, UTC-09:00]",[Oceania],"{'png': 'https://flagcdn.com/w320/pf.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [-17.53, -149.56]}",NaN,NaN,NaN,NaN


In [72]:
df_resties_mod['Currencies'].dropna()

0          {'XPF': {'name': 'CFP franc', 'symbol': '₣'}}
1               {'EUR': {'name': 'Euro', 'symbol': '€'}}
2      {'VES': {'name': 'Venezuelan bolívar soberano'...
3               {'EUR': {'name': 'Euro', 'symbol': '€'}}
4      {'USD': {'name': 'United States dollar', 'symb...
                             ...                        
245    {'USD': {'name': 'United States dollar', 'symb...
246    {'BWP': {'name': 'Botswana pula', 'symbol': 'P'}}
247    {'PAB': {'name': 'Panamanian balboa', 'symbol'...
248    {'XAF': {'name': 'Central African CFA franc', ...
249    {'USD': {'name': 'United States dollar', 'symb...
Name: Currencies, Length: 247, dtype: object

In [73]:
# Função para extrair a chave 'name' do dicionário
def extrair_nome_moeda(dicionario):
    if isinstance(dicionario, dict):
        for chave, valor in dicionario.items():
            if 'name' in valor:
                return valor['name']
    return None
        

# Extrair e atribuir a chave 'name' à coluna 'currencies'
df_resties_mod['Currencies'] = df_resties_mod['Currencies'].apply(extrair_nome_moeda)

# Renomear a coluna para 'Currencies_Name'
df_resties_mod = df_resties_mod.rename(columns={'Currencies': 'Currencies_Name'})


In [74]:
df_resties_mod.head(1)

,Name,Tld,Cca2,Ccn3,Cca3,Independent,Status,Unmember,Currencies_Name,Idd,Capital,Altspellings,Region,Subregion,Languages,Translations,Latlng,Landlocked,Area,Demonyms,Flag,Maps,Population,Car,Timezones,Continents,Flags,Coatofarms,Startofweek,Capitalinfo,Borders,Cioc,Gini,Fifa
0,French Polynesia,[.pf],PF,258,PYF,False,officially-assigned,False,CFP franc,"{'root': '+6', 'suffixes': ['89']}",[Papeetē],"[PF, Polynésie française, French Polynesia, Pō...",Oceania,Polynesia,{'fra': 'French'},"{'ara': {'official': 'بولينزيا الفرنسية', 'com...","[17.6797, 149.4068]",False,4167.0,"{'eng': {'f': 'French Polynesian', 'm': 'Frenc...",🇵🇫,{'googleMaps': 'https://goo.gl/maps/xgg6BQTRye...,280904,"{'signs': ['F'], 'side': 'right'}","[UTC-10:00, UTC-09:30, UTC-09:00]",[Oceania],"{'png': 'https://flagcdn.com/w320/pf.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [-17.53, -149.56]}",NaN,NaN,NaN,NaN


In [75]:
df_resties_mod['Borders'].fillna('NA', inplace=True)

In [76]:
colunas_com_listas = df_resties_mod.columns[df_resties_mod.applymap(type).eq(list).any()]

C:\Users\junio\AppData\Local\Temp\ipykernel_13936\2574065366.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  colunas_com_listas = df_resties_mod.columns[df_resties_mod.applymap(type).eq(list).any()]


In [77]:
colunas_com_listas

Index(['Tld', 'Capital', 'Altspellings', 'Latlng', 'Timezones', 'Continents',
       'Borders'],
      dtype='object')

In [78]:
for colunas in colunas_com_listas:
    df_resties_mod[colunas] = df_resties_mod[colunas].apply(json.dumps)

In [80]:
df_resties_mod['Continents'] = df_resties_mod['Continents'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[2:-2])

In [79]:
df_resties_mod['Capital'] = df_resties_mod['Capital'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[2:-2])

In [81]:
df_resties_mod['Latlng'] = df_resties_mod['Latlng'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[1:-1])

In [82]:
df_resties_mod['Borders'] = df_resties_mod['Borders'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[1:-1])

In [83]:
df_resties_mod['Timezones'] = df_resties_mod['Timezones'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[1:-1])

In [84]:
df_resties_mod['Capital'] = df_resties_mod['Capital'].str.strip('"')

In [185]:
df_resties_mod = df_resties_mod.drop(columns=['Translations','Demonyms','Languages','Gini','Capitalinfo','Idd','Car','Tld','Altspellings','Maps','Flags','Coatofarms'])

In [85]:

def converter_dict_para_list(value):
    if isinstance(value, dict):
        return list(value)  # Converte o dicionário em uma string
    else:
        return value  # Mantém outros valores inalterados


In [86]:
df_resties_mod['Timezones'] = df_resties_mod['Timezones'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[1:-1])

In [87]:
df_resties_mod.head(2)

,Name,Tld,Cca2,Ccn3,Cca3,Independent,Status,Unmember,Currencies_Name,Idd,Capital,Altspellings,Region,Subregion,Languages,Translations,Latlng,Landlocked,Area,Demonyms,Flag,Maps,Population,Car,Timezones,Continents,Flags,Coatofarms,Startofweek,Capitalinfo,Borders,Cioc,Gini,Fifa
0,French Polynesia,"["".pf""]",PF,258,PYF,False,officially-assigned,False,CFP franc,"{'root': '+6', 'suffixes': ['89']}",Papeet\u0113,"[""PF"", ""Polyn\u00e9sie fran\u00e7aise"", ""Frenc...",Oceania,Polynesia,{'fra': 'French'},"{'ara': {'official': 'بولينزيا الفرنسية', 'com...","17.6797, 149.4068",False,4167.0,"{'eng': {'f': 'French Polynesian', 'm': 'Frenc...",🇵🇫,{'googleMaps': 'https://goo.gl/maps/xgg6BQTRye...,280904,"{'signs': ['F'], 'side': 'right'}","UTC-10:00"", ""UTC-09:30"", ""UTC-09:00",Oceania,"{'png': 'https://flagcdn.com/w320/pf.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [-17.53, -149.56]}",NA,NaN,NaN,NaN
1,Saint Martin,"["".fr"", "".gp""]",MF,663,MAF,False,officially-assigned,False,Euro,"{'root': '+5', 'suffixes': ['90']}",Marigot,"[""MF"", ""Collectivity of Saint Martin"", ""Collec...",Americas,Caribbean,{'fra': 'French'},"{'ara': {'official': 'سانت مارتن', 'common': '...","18.0708, 63.0501",False,53.0,"{'eng': {'f': 'Saint Martin Islander', 'm': 'S...",🇲🇫,{'googleMaps': 'https://goo.gl/maps/P9ho9QuJ9E...,38659,"{'signs': ['F'], 'side': 'right'}",UTC-04:00,North America,"{'png': 'https://flagcdn.com/w320/mf.png', 'sv...",{},monday,"{'latlng': [18.07, -63.08]}","""SXM""",NaN,NaN,NaN


In [88]:
df_resties_mod['Cioc'] = df_resties_mod['Cioc'].fillna('Not_Apply')

In [89]:
valores_ausentes_por_coluna = df_resties_mod.isna().sum()

In [90]:
valores_ausentes_por_coluna

Name                0
Tld                 0
Cca2                0
Ccn3                1
Cca3                0
Independent         1
Status              0
Unmember            0
Currencies_Name     3
Idd                 0
Capital             0
Altspellings        0
Region              0
Subregion           5
Languages           1
Translations        0
Latlng              0
Landlocked          0
Area                0
Demonyms            1
Flag                0
Maps                0
Population          0
Car                 0
Timezones           0
Continents          0
Flags               0
Coatofarms          0
Startofweek         0
Capitalinfo         0
Borders             0
Cioc                0
Gini               83
Fifa               44
dtype: int64

In [91]:
df_resties_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             250 non-null    object 
 1   Tld              250 non-null    object 
 2   Cca2             250 non-null    object 
 3   Ccn3             249 non-null    object 
 4   Cca3             250 non-null    object 
 5   Independent      249 non-null    object 
 6   Status           250 non-null    object 
 7   Unmember         250 non-null    bool   
 8   Currencies_Name  247 non-null    object 
 9   Idd              250 non-null    object 
 10  Capital          250 non-null    object 
 11  Altspellings     250 non-null    object 
 12  Region           250 non-null    object 
 13  Subregion        245 non-null    object 
 14  Languages        249 non-null    object 
 15  Translations     250 non-null    object 
 16  Latlng           250 non-null    object 
 17  Landlocked      

In [92]:
mapeamento = {True: 'Yes', False: 'No'}

In [93]:
colunas_a_tratar = ['Independent', 'Unmember']
df_resties_mod[colunas_a_tratar] = df_resties_mod[colunas_a_tratar].replace(mapeamento)

In [94]:
df_resties_mod.head(2)

,Name,Tld,Cca2,Ccn3,Cca3,Independent,Status,Unmember,Currencies_Name,Idd,Capital,Altspellings,Region,Subregion,Languages,Translations,Latlng,Landlocked,Area,Demonyms,Flag,Maps,Population,Car,Timezones,Continents,Flags,Coatofarms,Startofweek,Capitalinfo,Borders,Cioc,Gini,Fifa
0,French Polynesia,"["".pf""]",PF,258,PYF,No,officially-assigned,No,CFP franc,"{'root': '+6', 'suffixes': ['89']}",Papeet\u0113,"[""PF"", ""Polyn\u00e9sie fran\u00e7aise"", ""Frenc...",Oceania,Polynesia,{'fra': 'French'},"{'ara': {'official': 'بولينزيا الفرنسية', 'com...","17.6797, 149.4068",False,4167.0,"{'eng': {'f': 'French Polynesian', 'm': 'Frenc...",🇵🇫,{'googleMaps': 'https://goo.gl/maps/xgg6BQTRye...,280904,"{'signs': ['F'], 'side': 'right'}","UTC-10:00"", ""UTC-09:30"", ""UTC-09:00",Oceania,"{'png': 'https://flagcdn.com/w320/pf.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [-17.53, -149.56]}",NA,Not_Apply,NaN,NaN
1,Saint Martin,"["".fr"", "".gp""]",MF,663,MAF,No,officially-assigned,No,Euro,"{'root': '+5', 'suffixes': ['90']}",Marigot,"[""MF"", ""Collectivity of Saint Martin"", ""Collec...",Americas,Caribbean,{'fra': 'French'},"{'ara': {'official': 'سانت مارتن', 'common': '...","18.0708, 63.0501",False,53.0,"{'eng': {'f': 'Saint Martin Islander', 'm': 'S...",🇲🇫,{'googleMaps': 'https://goo.gl/maps/P9ho9QuJ9E...,38659,"{'signs': ['F'], 'side': 'right'}",UTC-04:00,North America,"{'png': 'https://flagcdn.com/w320/mf.png', 'sv...",{},monday,"{'latlng': [18.07, -63.08]}","""SXM""",Not_Apply,NaN,NaN


SALVANDO E ENVIANDO O DF PARA O BANCO DE DADOS LOCAL

In [98]:

salva_bd(df_resties_mod,'resties')

ProgrammingError: Error binding parameter 10: type 'dict' is not supported

In [97]:
lista_tabelas()

,name
0,restaurantes
1,bancos
2,corretoras
3,resties


In [99]:
resties_teste = carrega_bd('resties')

In [100]:
resties_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Name             0 non-null      object
 1   Tld              0 non-null      object
 2   Cca2             0 non-null      object
 3   Ccn3             0 non-null      object
 4   Cca3             0 non-null      object
 5   Independent      0 non-null      object
 6   Status           0 non-null      object
 7   Unmember         0 non-null      object
 8   Currencies_Name  0 non-null      object
 9   Idd              0 non-null      object
 10  Capital          0 non-null      object
 11  Altspellings     0 non-null      object
 12  Region           0 non-null      object
 13  Subregion        0 non-null      object
 14  Languages        0 non-null      object
 15  Translations     0 non-null      object
 16  Latlng           0 non-null      object
 17  Landlocked       0 non-null      object
 18  Ar